In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from AlgoTestFactory import *

# altf1 = AlgoTestFactory([0, 10000], 15, savePath="Combined_4_test2", storeAccuracies=True)

# altf1.run()

In [3]:
from SktWsegRWR_utf8 import *
import pickle
import ProbData
from ProbModels import *
import multiprocessing
import math
import json
import pprint

Dataloader Started[Prob]...
Dataloader Finished[Prob]...


In [4]:
pb = ProbModels(fullCo_oc_mat = ProbData.fullCo_oc_mat, unigram_counts = ProbData.unigram_counts,
               cng2cngFullMat = ProbData.cng2cngFullMat, cng2index_dict = ProbData.cng2index_dict,
               w2w_samecng_fullmat=ProbData.w2w_samecng_fullmat, samecng_unigram_counts=ProbData.samecng_unigram_counts,
               v2c_fullMat = ProbData.v2c_fullMat)

goodFileDict = pickle.load(open('mergedGood_v3.p', 'rb'))
algo = SktWsegRWR(
            w2w_modelFunc = pb.get_w2w_mat, 
            t2t_modelFunc = pb.get_cng2cng_mat,
            v2c_modelFunc = pb.get_v2c_ranking,
            sameCng_modelFunc = pb.get_w2w_samecng_mat,
            partition=[0.25, 0.25, 0.25, 0.1]
        )

In [5]:
def loadSentence(fName, folderTag):
        # print('File: ', fName)
        try:
            dcsObj = pickleFixLoad('../Text Segmentation/DCS_pick/' + fName)           
            if folderTag == "C1020" :
                sentenceObj = pickleFixLoad('../TextSegmentation/corrected_10to20/' + fName)
            else:
                sentenceObj = pickleFixLoad('../TextSegmentation/Pickle_Files/' + fName)

        except (KeyError, EOFError, pickle.UnpicklingError) as e:
            return None, None
        return(sentenceObj, dcsObj)
    
def Accuracy(prediction, dcsObj):
#     solution = [rom_slp(c) for c in dcsObj.dcs_chunks]
    solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#     ac = 100*sum(list(map(lambda x: x in solution, prediction)))/len(prediction)
    ac = 100*sum(list(map(lambda x: x in prediction, solution)))/len(solution)
    return ac

In [7]:
# lowScore = lowScore[0:10]
# midScore = midScore[0:10]
# highScore = highScore[0:10]
# print(len(lowScore))
# print(lowScore)
# print(len(midScore))
# print(midScore)
# print(len(highScore))
# print(highScore)

In [ ]:
np.set_printoptions(precision=6)
jsonBatch = []
# for f in list(goodFileDict.keys())[1:10]:
# for f in list(goodFileDict.keys())[1:100]:
for f_ in lowScore:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        if(result != None):
            ac = runDetails['accuracy']
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#             print(result)
#             print('vs')
#             print(solution)
#             print(f, "\t", ac)
            jsonBatch.append(runDetails)
pickle.dump(jsonBatch, open('.temp/lowScore.p','wb'))
#################################################################
jsonBatch = []
# for f in list(goodFileDict.keys())[1:10]:
# for f in list(goodFileDict.keys())[1:100]:
for f_ in midScore:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        if(result != None):
            ac = runDetails['accuracy']
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#             print(result)
#             print('vs')
#             print(solution)
#             print(f, "\t", ac)
            jsonBatch.append(runDetails)
pickle.dump(jsonBatch, open('.temp/midScore.p','wb'))
#################################################################
jsonBatch = []
# for f in list(goodFileDict.keys())[1:10]:
# for f in list(goodFileDict.keys())[1:100]:
for f_ in highScore:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        if(result != None):
            ac = runDetails['accuracy']
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#             print(result)
#             print('vs')
#             print(solution)
#             print(f, "\t", ac)
            jsonBatch.append(runDetails)
print(pprint.pprint(jsonBatch))
pickle.dump(jsonBatch, open('.temp/highScore.p','wb'))

In [33]:
np.set_printoptions(precision=6)
d = pickle.load(open('.temp/midScore.p','rb'), encoding='utf8')
f = d[0]
print(f.keys())
print()
print(f['sentences'])
print(f['0'].keys())
print(f['initialQuery'])
print(f['0']['updated_query'])
print(f['0']['final_score'])
print(np.asarray(f['0']['final_score'].argsort()))
print()
print(f['6']['updated_query'])
print(f['6']['final_score'])

dict_keys(['5', 'TransitionMat_t2t', 'initialQuery', 'prediction', '3', 'TransitionMat_w2w_samecng', '4', 'DCSLemmas', 'accuracy', 'TransitionMat_w2w', '2', 'nodeList', '1', '0', 'sentence', '6'])

dict_keys(['updated_query', 'final_score', 'removed', 'final_rank', 'w2w_samecng_rank', 'w2w_samecng_score', 'w2w_rank', 't2t_score', 't2t_rank', 'w2w_score', 'winner'])
[15, 16]
[15, 16, 21]
[[ 0.040709  0.04046   0.041591  0.041591  0.037939  0.041064  0.041101
   0.041317  0.041317  0.041353  0.041227  0.041433  0.041429  0.041316
   0.040558  0.031458  0.03466   0.041182  0.041182  0.041182  0.040675
   0.037607]]
[[15 16 21  4  1 14 20  0  5  6 17 18 19 10 13  8  7  9 12 11  2  3]]

[15, 16, 21, 4, 1, 20, 14, 10, 2]
[[ 0.040852  0.038907  0.040852  0.040852  0.038907  0.040852  0.040852
   0.040852  0.040852  0.040852  0.038907  0.040852  0.040852  0.040852
   0.038907  0.038072  0.038907  0.040852  0.040852  0.040852  0.038907
   0.038907]]


In [151]:
np.set_printoptions(precision=4)
for f in list(goodFileDict.keys())[1:10]:
# for f in list(goodFileDict.keys())[1:100]:
# for f in fullfs[0:1]:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        if(result != None):
            ac = runDetails['accuracy']
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#             print(result)
#             print('vs')
#             print(solution)
            print(f, "\t", ac)
#             pprint.pprint(runDetails)

435293.p 	 100.0
305404.p 	 71.42857142857143
396827.p 	 100.0
18144.p 	 83.33333333333333


In [131]:
np.set_printoptions(precision=6)
# for f in list(goodFileDict.keys())[90:91]:
for f in list(goodFileDict.keys())[1:100]:
# for f in fullfs:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj)
        if(result != None):
            ac = Accuracy(result, dcsObj)
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
#             print(result)
#             print('vs')
#             print(solution)
#             print(f, "\t", ac)

0 [0]
1 [0]
Chosen: 1


UnboundLocalError: local variable 'stepDetails' referenced before assignment

In [151]:
fullfs = ['434538.p','148267.p','379245.p']

In [162]:
lowScore = []
midScore = []
highScore = []
allOfEm = []
np.set_printoptions(precision=4)
# for f in list(goodFileDict.keys())[1:100]:
# for f in list(goodFileDict.keys())[5:6]:
for f in fullfs[0:1]:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        
        if(result != None):
#             print(runDetails['nodeList'])
            ac = runDetails['accuracy']
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]

            print(f, "\t", ac)
#             if(ac <= 100):
#                 print(result)
#                 print('vs')
#                 print(solution)
#                 pprint.pprint(runDetails['nodeList'])
#                 break
            # CODE FOR PRINTING RUNDETAILS
#             pprint.pprint(runDetails['nodeList'])
#             for i in range(runDetails['steps']):
#                 if str(i) in runDetails:
#                     print('Step', i)
#                     step = runDetails[str(i)]
#                     pprint.pprint(step['winner'])

            # CODE FOR SELECTING FILES 
#             allOfEm.append((f, ac))
#             if(ac <= 40):
#                 lowScore.append((f, ac))
#             elif (ac <= 60):
#                 if(ac >= 50):
#                     midScore.append((f, ac))
#             elif (ac <= 100):
#                 if(ac >= 80):
#                     highScore.append((f, ac))

# print(len(allOfEm))
# print(len(lowScore))
# print(lowScore)
# print(len(midScore))
# print(midScore)
# print(len(highScore))
# print(highScore)


Remvoe b/w mamatvena mamatva
Remvoe b/w mamatvena mamatve
Remvoe b/w mamatvena mama
Remvoe b/w mamatvena mama
Remvoe b/w mamatva mamatve
Remvoe b/w mamatva mama
Remvoe b/w mamatva mama
Remvoe b/w mamatve mama
Remvoe b/w mamatve mama
Remvoe b/w mama mama
Remvoe b/w tvA tu


IndexError: list index out of range

In [135]:
SeeSentence(sentenceObj)

mamatvenABiBUtaH  sa tatrEva parivartate   
Analyzing  mamatvenABiBUtaḥ
0 :  mamatvena ['mamatva'] [{'noun': ['i. sg. n.']}]
0 :  mamatva ['mamatva'] [{'compound': ['iic.']}]
0 :  mamatve ['mamatva'] [{'noun': ['acc. du. n.', 'nom. du. n.', 'loc. sg. n.']}]
0 :  mama ['asmad'] [{'noun': ['g. sg. *']}]
0 :  mama ['mA_1', 'mA_4'] [{'verb': ['pft. ac. pl. 2']}, {'verb': ['pft. ac. pl. 2']}]
4 :  tvA ['yuzmad'] [{'noun': ['acc. sg. *']}]
4 :  tu ['tu'] [{'indeclinable': ['conj.']}]
6 :  ina ['na'] [{'compound': ['iic.']}]
6 :  inA ['na'] [{'noun': ['nom. sg. f.']}]
6 :  enA ['enA_2'] [{'undetermined': ['adv.']}]
7 :  nABi ['nAB'] [{'compound': ['iic.']}]
7 :  na ['na'] [{'indeclinable': ['part.']}]
8 :  aBi ['aB'] []
8 :  aBi ['aB'] [{'indeclinable': ['prep.']}]
8 :  aBi ['aB'] [{'compound': ['iic.']}]
11 :  BUtas ['BUta', 'BU_1'] [{'noun': ['nom. sg. m.']}, {'verb': ['pp.']}]
11 :  BU ['BU_2'] [{'compound': ['iic.']}]
11 :  Bu ['Bu', 'BU_2'] [{'indeclinable': ['ind.']}, {'indeclinable': [

In [100]:
SeeDCS(dcsObj)

mamatvenABiBUtaH  sa tatrEva parivartate   
[['mamatva', 'abhibhū'], ['tad'], ['tatra', 'eva'], ['parivṛt']]
Lemmas: ['mamatva', 'aBiBU', 'tad', 'tatra', 'eva', 'parivft']
[['91', '-190'], ['29'], ['2', '2'], ['-13']]


In [203]:
list(goodFileDict.keys())[13]

'327359.p'

In [209]:
lowfs

['179084.p',
 '212470.p',
 '141314.p',
 '312382.p',
 '371565.p',
 '20967.p',
 '178347.p',
 '289080.p',
 '334071.p',
 '407484.p',
 '209444.p']

In [ ]:
# 7941
# Results: 
# Mean:  65.7963242743
# Percentiles:  [   5.88   50.     66.67   85.71  100.  ]
